In [1]:
# 국토교통부 버스노선(서울, 부산 제외)
import requests, bs4
import mysql.connector
from lxml import html
from urllib.request import Request,urlopen
from urllib.parse import urlencode,quote_plus
from bs4 import BeautifulSoup
from xml.etree.ElementTree import parse
from ServiceKey import public_serviceKey,mySQL_Connect

mydb = mySQL_Connect()

cursor = mydb.cursor(prepared=True)
citycode = '36430'

# DB에서 도시 코드 구해오기
# 2021.09.26 데이터 삽입완료
query = ("select citycode from public_busctycodelist where citycode >= {}".format(citycode))

cursor.execute(query)

location_list = []

for i in cursor:
    location_list.append(i)
    
b_cnt = 0
for j in location_list:
    location = j
    if b_cnt == 1:
        mydb.close()
        break
    
    for i in range(1,121):
        routeNo = i
        print("location : {} | {}번째 조회시작".format(location[0],routeNo))    
        url = 'http://openapi.tago.go.kr/openapi/service/BusRouteInfoInqireService/getRouteNoList'
        queryParams = '?' + urlencode({
            quote_plus('serviceKey') : public_serviceKey(),
            quote_plus('cityCode') : location[0],
            quote_plus('routeNo') : routeNo,
            quote_plus('numOfRows') : '10000'
        })

        response = requests.get(url + queryParams).text.encode('utf-8')
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

        rows = xmlobj.findAll('item')
        resultCode = xmlobj.findAll('resultCode')
        if resultCode[0].text == '99':
            print("location : {} | {} 번째부터 조회수 제한이 걸렸습니다.".format(location[0],routeNo))
            b_cnt += 1
            break
        if rows == []:
            print("{} 번쨰데이터가없습니다.".format(routeNo))
            continue
        for j in rows:
            item = j
            routeno = item.findAll('routeno')
            if routeno == []:
                routeno = None
            else:
                routeno = routeno[0].text

            routeid = item.findAll('routeid')
            if routeid == []:
                routeid = None
            else:
                routeid = routeid[0].text

            routetp = item.findAll('routetp')
            if routetp == []:
                routetp = None
            else:
                routetp = routetp[0].text

            startnodenm = item.findAll('startnodenm')
            if startnodenm == []:
                startnodenm = None
            else:
                startnodenm = startnodenm[0].text

            startvehicletime = item.findAll('startvehicletime')
            if startvehicletime == []:
                startvehicletime = None
            else:
                startvehicletime = startvehicletime[0].text

            endnodenm = item.findAll('endnodenm')
            if endnodenm == []:
                endnodenm = None
            else:
                endnodenm = endnodenm[0].text

            endvehicletime = item.findAll('endvehicletime')
            if endvehicletime == []:
                endvehicletime = None
            else:
                endvehicletime = endvehicletime[0].text
                
            sql = "INSERT INTO public_busroute_info (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime,citycode) VALUES (?,?,?,?,?,?,?,?)"
            val = (routeno, routeid, routetp, startnodenm, startvehicletime, endnodenm, endvehicletime, location[0])


            cursor.execute(sql, val)
            mydb.commit()

location : 36020 | 1번째 조회시작
location : 36020 | 2번째 조회시작
location : 36020 | 3번째 조회시작
location : 36020 | 4번째 조회시작
location : 36020 | 5번째 조회시작
location : 36020 | 6번째 조회시작
location : 36020 | 7번째 조회시작
location : 36020 | 8번째 조회시작
location : 36020 | 9번째 조회시작
location : 36020 | 10번째 조회시작
location : 36020 | 11번째 조회시작
location : 36020 | 12번째 조회시작
location : 36020 | 13번째 조회시작
location : 36020 | 14번째 조회시작
location : 36020 | 15번째 조회시작
location : 36020 | 16번째 조회시작
location : 36020 | 17번째 조회시작
17 번쨰데이터가없습니다.
location : 36020 | 18번째 조회시작
18 번쨰데이터가없습니다.
location : 36020 | 19번째 조회시작
19 번쨰데이터가없습니다.
location : 36020 | 20번째 조회시작
location : 36020 | 21번째 조회시작
location : 36020 | 22번째 조회시작
location : 36020 | 23번째 조회시작
location : 36020 | 24번째 조회시작
location : 36020 | 25번째 조회시작
location : 36020 | 26번째 조회시작
location : 36020 | 27번째 조회시작
location : 36020 | 28번째 조회시작
location : 36020 | 29번째 조회시작
location : 36020 | 30번째 조회시작
location : 36020 | 31번째 조회시작
location : 36020 | 32번째 조회시작
location : 36020 | 33번째 조회시작
locatio

110 번쨰데이터가없습니다.
location : 36030 | 111번째 조회시작
location : 36030 | 112번째 조회시작
112 번쨰데이터가없습니다.
location : 36030 | 113번째 조회시작
113 번쨰데이터가없습니다.
location : 36030 | 114번째 조회시작
114 번쨰데이터가없습니다.
location : 36030 | 115번째 조회시작
115 번쨰데이터가없습니다.
location : 36030 | 116번째 조회시작
116 번쨰데이터가없습니다.
location : 36030 | 117번째 조회시작
117 번쨰데이터가없습니다.
location : 36030 | 118번째 조회시작
118 번쨰데이터가없습니다.
location : 36030 | 119번째 조회시작
119 번쨰데이터가없습니다.
location : 36030 | 120번째 조회시작
120 번쨰데이터가없습니다.
location : 36060 | 1번째 조회시작
location : 36060 | 2번째 조회시작
location : 36060 | 3번째 조회시작
location : 36060 | 4번째 조회시작
location : 36060 | 5번째 조회시작
location : 36060 | 6번째 조회시작
location : 36060 | 7번째 조회시작
location : 36060 | 8번째 조회시작
location : 36060 | 9번째 조회시작
location : 36060 | 10번째 조회시작
location : 36060 | 11번째 조회시작
location : 36060 | 12번째 조회시작
location : 36060 | 13번째 조회시작
13 번쨰데이터가없습니다.
location : 36060 | 14번째 조회시작
14 번쨰데이터가없습니다.
location : 36060 | 15번째 조회시작
location : 36060 | 16번째 조회시작
16 번쨰데이터가없습니다.
location : 36060 | 17번째 조회시작
17 번쨰데이터가없습

71 번쨰데이터가없습니다.
location : 36330 | 72번째 조회시작
72 번쨰데이터가없습니다.
location : 36330 | 73번째 조회시작
73 번쨰데이터가없습니다.
location : 36330 | 74번째 조회시작
74 번쨰데이터가없습니다.
location : 36330 | 75번째 조회시작
75 번쨰데이터가없습니다.
location : 36330 | 76번째 조회시작
76 번쨰데이터가없습니다.
location : 36330 | 77번째 조회시작
77 번쨰데이터가없습니다.
location : 36330 | 78번째 조회시작
78 번쨰데이터가없습니다.
location : 36330 | 79번째 조회시작
79 번쨰데이터가없습니다.
location : 36330 | 80번째 조회시작
80 번쨰데이터가없습니다.
location : 36330 | 81번째 조회시작
81 번쨰데이터가없습니다.
location : 36330 | 82번째 조회시작
82 번쨰데이터가없습니다.
location : 36330 | 83번째 조회시작
83 번쨰데이터가없습니다.
location : 36330 | 84번째 조회시작
84 번쨰데이터가없습니다.
location : 36330 | 85번째 조회시작
85 번쨰데이터가없습니다.
location : 36330 | 86번째 조회시작
86 번쨰데이터가없습니다.
location : 36330 | 87번째 조회시작
87 번쨰데이터가없습니다.
location : 36330 | 88번째 조회시작
88 번쨰데이터가없습니다.
location : 36330 | 89번째 조회시작
89 번쨰데이터가없습니다.
location : 36330 | 90번째 조회시작
90 번쨰데이터가없습니다.
location : 36330 | 91번째 조회시작
91 번쨰데이터가없습니다.
location : 36330 | 92번째 조회시작
92 번쨰데이터가없습니다.
location : 36330 | 93번째 조회시작
93 번쨰데이터가없습니다.
location : 36330 

42 번쨰데이터가없습니다.
location : 36400 | 43번째 조회시작
location : 36400 | 44번째 조회시작
44 번쨰데이터가없습니다.
location : 36400 | 45번째 조회시작
45 번쨰데이터가없습니다.
location : 36400 | 46번째 조회시작
location : 36400 | 47번째 조회시작
location : 36400 | 48번째 조회시작
48 번쨰데이터가없습니다.
location : 36400 | 49번째 조회시작
49 번쨰데이터가없습니다.
location : 36400 | 50번째 조회시작
location : 36400 | 51번째 조회시작
location : 36400 | 52번째 조회시작
location : 36400 | 53번째 조회시작
location : 36400 | 54번째 조회시작
54 번쨰데이터가없습니다.
location : 36400 | 55번째 조회시작
location : 36400 | 56번째 조회시작
location : 36400 | 57번째 조회시작
location : 36400 | 58번째 조회시작
location : 36400 | 59번째 조회시작
location : 36400 | 60번째 조회시작
location : 36400 | 61번째 조회시작
location : 36400 | 62번째 조회시작
location : 36400 | 63번째 조회시작
location : 36400 | 64번째 조회시작
64 번쨰데이터가없습니다.
location : 36400 | 65번째 조회시작
location : 36400 | 66번째 조회시작
66 번쨰데이터가없습니다.
location : 36400 | 67번째 조회시작
location : 36400 | 68번째 조회시작
68 번쨰데이터가없습니다.
location : 36400 | 69번째 조회시작
location : 36400 | 70번째 조회시작
location : 36400 | 71번째 조회시작
location : 36400 | 72번째 

36 번쨰데이터가없습니다.
location : 36420 | 37번째 조회시작
37 번쨰데이터가없습니다.
location : 36420 | 38번째 조회시작
38 번쨰데이터가없습니다.
location : 36420 | 39번째 조회시작
39 번쨰데이터가없습니다.
location : 36420 | 40번째 조회시작
location : 36420 | 41번째 조회시작
41 번쨰데이터가없습니다.
location : 36420 | 42번째 조회시작
location : 36420 | 43번째 조회시작
43 번쨰데이터가없습니다.
location : 36420 | 44번째 조회시작
44 번쨰데이터가없습니다.
location : 36420 | 45번째 조회시작
location : 36420 | 46번째 조회시작
46 번쨰데이터가없습니다.
location : 36420 | 47번째 조회시작
47 번쨰데이터가없습니다.
location : 36420 | 48번째 조회시작
48 번쨰데이터가없습니다.
location : 36420 | 49번째 조회시작
49 번쨰데이터가없습니다.
location : 36420 | 50번째 조회시작
location : 36420 | 51번째 조회시작
51 번쨰데이터가없습니다.
location : 36420 | 52번째 조회시작
52 번쨰데이터가없습니다.
location : 36420 | 53번째 조회시작
53 번쨰데이터가없습니다.
location : 36420 | 54번째 조회시작
54 번쨰데이터가없습니다.
location : 36420 | 55번째 조회시작
location : 36420 | 56번째 조회시작
56 번쨰데이터가없습니다.
location : 36420 | 57번째 조회시작
57 번쨰데이터가없습니다.
location : 36420 | 58번째 조회시작
58 번쨰데이터가없습니다.
location : 36420 | 59번째 조회시작
59 번쨰데이터가없습니다.
location : 36420 | 60번째 조회시작
location : 36420 | 